<div class="min-h-screen bg-neutral-900 text-white">
  <div class="max-w-3xl mx-auto py-10 px-4">
    <h1 class="text-3xl font-bold text-center mb-8">Bathroom Statistics</h1>
    <div class="overflow-x-auto">
      <table class="min-w-full border border-white text-sm text-left">
        <thead class="text-white">
          <tr>
            <th class="px-4 py-2 border border-white bg-[#404040] text-center">Statistic</th>
            <th class="px-4 py-2 border border-white bg-[#404040] text-center">Value</th>
          </tr>
        </thead>
        <tbody style="background-color: #404040;">
          <tr>
            <td class="border border-white px-4 py-2 text-center">Average Duration (minutes)</td>
            <td class="border border-white px-4 py-2 text-center" id="avg-duration">placeholder</td>
          </tr>
          <tr>
            <td class="border border-white px-4 py-2 text-center">Number of Times Gone</td>
            <td class="border border-white px-4 py-2 text-center" id="num-times">placeholder</td>
          </tr>
        </tbody>
      </table>
    </div>
  </div>
</div>

<script type="module">
  import { pythonURI, javaURI, fetchOptions } from '{{site.baseurl}}/assets/js/api/config.js';

  function calculateAverageDuration(timeIn) {
    const visits = timeIn.split(',');
    let totalDuration = 0;
    visits.forEach(visit => {
      const [checkIn, checkOut] = visit.split('-');
      const checkInTime = new Date('1970-01-01T' + checkIn + 'Z').getTime();
      const checkOutTime = new Date('1970-01-01T' + checkOut + 'Z').getTime();
      const duration = (checkOutTime - checkInTime) / 1000 / 60;
      totalDuration += duration;
    });
    return totalDuration / visits.length;
  }

  function getTinkle(personName) {
    const tinkleURL = javaURI + `/api/tinkle/${personName}`;
    fetch(tinkleURL, { ...fetchOptions, credentials: 'include' })
      .then(response => {
        if (!response.ok) {
          console.error("HTTP status code: " + response.status);
          return null;
        }
        return response.json();
      })
      .then(data => {
        if (!data) return;

        const timeIn = data.timeIn;
        const numVisits = timeIn.split(',').length;
        document.getElementById('num-times').textContent = numVisits;

        const avgDuration = calculateAverageDuration(timeIn);
        document.getElementById('avg-duration').textContent = avgDuration.toFixed(2);
      })
      .catch(err => console.error("Fetch error: ", err));
  }

  function getPerson() {
    const URL = javaURI + '/api/person/get';
    fetch(URL, { ...fetchOptions, credentials: 'include' })
      .then(response => {
        if (!response.ok) {
          console.error("HTTP status code: " + response.status);
          return null;
        }
        return response.json();
      })
      .then(data => {
        if (!data) return;
        getTinkle(encodeURIComponent(data.name));
      })
      .catch(err => console.error("Fetch error: ", err));
  }

  window.onload = function () {
    getPerson();
  };
</script>